# Debug, Evaluate, and Monitor LLMs with LangSmith

LangChain makes it easy to get started with Agents and other LLM applications. Even so, delivering a high-quality agent to production can be deceptively difficult. You will have to customize prompts, data, llms, and more to make it higher quality. To aid the development process, we've designed call tracing at the core of LangChain. In this notebook, you will get started prototyping, testing, and monitoring an LLM agent.

When might you want to use run tracing? Some situations we've found it useful include:
- Comparing prompts
- Comparing LLMs
- Quickly debugging a new chain, agent, or set of tools
- Running a given chain multiple time on a dataset to ensure it consistently meets a quality bar.

## Prerequisites

**Either [create a hosted LangSmith account](https://www.langchain.plus/) and connect with an API key OR
[run the server locally](https://docs.smith.langchain.com/docs/additional-resources/local_installation).**


To run the local server, execute the following comand in your terminal:
```
pip install --upgrade langchain
langchain plus start
```

Now, let's get started by configuring our environment.

## Debug your Agent

First, configure your environment variables to tell LangChain to log traces. This is done by setting the `LANGCHAIN_TRACING_V2` environment variable to true.
You can tell LangChain which project to log to by setting the `LANGCHAIN_PROJECT` environment variable. This will automatically create a debug project for you.

For more information on other ways to set up tracing, please reference the [LangSmith documentation](https://docs.langchain.plus/docs/)

**NOTE:** You must also set your `OPENAI_API_KEY` and `SERPAPI_API_KEY` environment variables in order to run the following tutorial.

**NOTE:** You can optionally set the `LANGCHAIN_ENDPOINT` and `LANGCHAIN_API_KEY` environment variables if using the hosted version.

In [1]:
import os
from uuid import uuid4
from langchainplus_sdk import LangChainPlusClient

unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {unique_id}"
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"  # Uncomment this line to use the hosted version
# os.environ["LANGCHAIN_API_KEY"] = "<YOUR-LANGSMITH-API-KEY>"  # Uncomment this line to use the hosted version.

# Used by the agent below
# os.environ["OPENAI_API_KEY"] = "<YOUR-OPENAI-API-KEY>"
# os.environ["SERPAPI_API_KEY"] = "<YOUR-SERPAPI-API-KEY>"

client = LangChainPlusClient()
print("You can click the link below to view the UI")
client

You can click the link below to view the UI


LangChainPlusClient (API URL: https://dev.api.smith.langchain.com)

Now, start prototyping your agent. We will use a straightforward math example.

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, load_tools
from langchain.agents import AgentType

llm = ChatOpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=False
)

In [3]:
import asyncio

inputs = [
    "How many people live in canada as of 2023?",
    "who is dua lipa's boyfriend? what is his age raised to the .43 power?",
    "what is dua lipa's boyfriend age raised to the .43 power?",
    "how far is it from paris to boston in miles",
    "what was the total number of points scored in the 2023 super bowl? what is that number raised to the .23 power?",
    "what was the total number of points scored in the 2023 super bowl raised to the .23 power?",
    "how many more points were scored in the 2023 super bowl than in the 2022 super bowl?",
    "what is 153 raised to .1312 power?",
    "who is kendall jenner's boyfriend? what is his height (in inches) raised to .13 power?",
    "what is 1213 divided by 4345?",
]
results = []


async def arun(agent, input_example):
    try:
        return await agent.arun(input_example)
    except Exception as e:
        # The agent sometimes makes mistakes! These will be captured by the tracing.
        return e


for input_example in inputs:
    results.append(arun(agent, input_example))
results = await asyncio.gather(*results)

In [4]:
from langchain.callbacks.tracers.langchain import wait_for_all_tracers

# Logs are submitted in a background thread. Make sure they've been submitted before moving on.
wait_for_all_tracers()

Assuming you've successfully initiated the server as described earlier, your agent logs should show up in your server. You can check by clicking on the link below:

In [5]:
client

LangChainPlusClient (API URL: https://dev.api.smith.langchain.com)

## Test

Once you've debugged a prototype of your agent, you will want to create tests and benchmark evaluations as you think about putting it into a production environment.

In this notebook, you will run evaluators to test an agent. You will do so in a few steps:

1. Create a dataset
2. Select or create evaluators to measure performance
3. Define the LLM or Chain initializer to test
4. Run the chain and evaluators using the helper functions

### 1. Create Dataset

Below, use the client to create a dataset from the Agent runs you just logged while debugging above. You will use these later to measure performance.

For more information on datasets, including how to create them from CSVs or other files or how to create them in the web app, please refer to the [LangSmith documentation](https://docs.smith.langchain.com/docs).

In [6]:
dataset_name = f"calculator-example-dataset-{unique_id}"

dataset = client.create_dataset(
    dataset_name, description="A calculator example dataset"
)

runs = client.list_runs(
    project_name=os.environ["LANGCHAIN_PROJECT"],
    execution_order=1,  # Only return the top-level runs
    error=False,  # Only runs that succeed
)
for run in runs:
    client.create_example(inputs=run.inputs, outputs=run.outputs, dataset_id=dataset.id)

### 2. Define the Agent or LLM to Test

You can evaluate any chain, agent, or LLM. Since chains can have memory, we need to pass an
initializer function that returns a new chain for each row.

In this case, you will test an agent that uses OpenAI's function calling endpoints, but it can be any simple chain.

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, load_tools, AgentType

llm = ChatOpenAI(model="gpt-3.5-turbo-0613", temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)


# Since chains can be stateful (e.g. they can have memory), we need provide
# a way to initialize a new chain for each row in the dataset. This is done
# by passing in a factory function that returns a new chain for each row.
def agent_factory():
    return initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=False)


# If your chain is NOT stateful, your factory can return the object directly
# to improve runtime performance. For example:
# chain_factory = lambda: agent

### 3. Select RunEvaluators

Manually comparing the results of chains in the UI is effective, but it can be time consuming.
It's easier to leverage AI-assisted feedback to evaluate your agent's performance.

Below, we will create some pre-implemented run evaluators that do the following:
- Compare results against ground truth labels. (You used the debug outputs above for this)
- Evaluate the overall agent trajectory based on the tool usage and intermediate steps.
- Use criteria to evaluate 'aspects' of the agent's response in a reference-free manner using custom criteria

For a longer discussion of how to select an appropriate evaluator for your use case and how to create your own
custom evaluators, please refer to the [LangSmith documentation](https://docs.langchain.plus/docs/).

Below, create the run evaluators.

**Note: the feedback API is currently experimental and subject to change.**

In [8]:
from langchain.evaluation import EvaluatorType
from langchain.evaluation.run_evaluators import (
    load_run_evaluators_for_model,
    load_run_evaluator_for_model,
)

model_to_evaluate = agent_factory()

# The QA 'correctness' evaluator requires a reference label from the dataset examples.
qa_evaluator = load_run_evaluator_for_model(EvaluatorType.QA, model_to_evaluate, reference_key="output")

# Measure helpfulness. We have some pre-defined criteria you can select
helpfulness_evaluator = load_run_evaluator_for_model(EvaluatorType.CRITERIA, model_to_evaluate, criteria="helpfulness")

# Custom criteria are specified as a dictionary
custom_criteria_evaluator = load_run_evaluator_for_model(
    EvaluatorType.CRITERIA,
    model_to_evaluate,
    criteria={
        "fifth-grader-score": "Do you have to be smarter than a fifth grader to answer this question?"
    })

# You can also use embedding distance or other non-LLM evaluators
embedding_evaluator = load_run_evaluator_for_model(
    EvaluatorType.EMBEDDING_DISTANCE,
    model_to_evaluate,
    reference_key="output",
)

# Measures how effective and efficient the agent's actions are
tools = load_tools(["serpapi", "llm-math"], llm=llm)
trajectory_evaluator = load_run_evaluator_for_model(EvaluatorType.AGENT_TRAJECTORY, model_to_evaluate, agent_tools=tools)


evaluators = [
    qa_evaluator,
    trajectory_evaluator,
    embedding_evaluator,
    helpfulness_evaluator,
    custom_criteria_evaluator,
]

### 4. Run the Agent and Evaluators

With the dataset, agent, and evaluators selected, you can use the helper function below to run them all.

The run traces and evaluation feedback will automatically be associated with the dataset for easy attribution and analysis.

In [9]:
from langchain.client import (
    arun_on_dataset,
    run_on_dataset,  # Available if your chain doesn't support async calls.
)

In [ ]:
chain_results = await arun_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=agent_factory,
    concurrency_level=5,  # Optional, sets the number of examples to run at a time
    verbose=True,
    client=client,
    tags=[
        "testing-notebook",
    ],  # Optional, adds a tag to the resulting chain runs
    run_evaluators=evaluators,
)

# Sometimes, the agent will error due to parsing issues, incompatible tool inputs, etc.
# These are logged as warnings here and captured as errors in the tracing UI.

Chain failed for example bbcff65a-f9d6-4b4a-9e7b-f3e626387411. Error: LLMMathChain._evaluate("
age_of_Dua_Lipa_boyfriend ** 0.43
") raised error: 'age_of_Dua_Lipa_boyfriend'. Please try again with a valid numerical expression


/Users/wfh/code/lc/lckg/langchain/callbacks/manager.py:167: UserWarning: The tracing v2 API is in development. This is not yet stable and may change in the future.
  warnings.warn(


Chain failed for example 5ca0058d-c7ec-4059-9894-0e200474c9dc. Error: Too many arguments to single-input tool Calculator. Args: ['height ^ 0.13', {'height': 68}]


/Users/wfh/code/lc/lckg/langchain/evaluation/schema.py:88: UserWarning: Ignoring input in EmbeddingDistanceEvalChain, as it is not expected.
  warn(self._skip_input_warning)


### Review the Test Results

You can review the test results tracing UI below by navigating to the Testing project 
with the title that starts with **"calculator-example-dataset-AgentExecutor-"**

This will show the new runs and the feedback logged from the selected evaluators.

In [ ]:
# You can navigate to the UI by clicking on the link below
client

For a real production application, you will want to add many more test cases and
incorporate larger datasets to run benchmark evaluations to measure aggregate performance
across. For more information on recommended ways to do this, see [LangSmith Documentation](https://docs.smith.langchain.com/docs/)

## Monitor

Once your agent passed the selected quality bar, you can deploy it to production. For this notebook, you will simulate user interactions directly while logging your traces to LangSmith for monitoring.

For more information on real production deployments, check out the [LangChain documentation](https://python.langchain.com/docs/guides/deployments/) or contact us at [support@langchain.dev](mailto:support@langchain.dev).

**First, create a new project to use in your production deployment.**

In [ ]:
deployment_name = f"Search + Calculator Deployment - {unique_id}"
project = client.create_project(deployment_name)

**Then, deploy your agent to production, making sure to configure the environment to log to the monitoring project.**

In [ ]:
agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=False)

In [ ]:
os.environ["LANGCHAIN_PROJECT"] = deployment_name

inputs = [
    "What's the ratio of the current US GDP to the average lifespan of a human?",
    "What's sin of 180 degrees?",
    "I need help on my homework",
    "If the price of bushel of wheat increases by 10 cents, about how much will that impact the average cost of bread?",
    # etc.
]
for query in inputs:
    try:
        await agent.arun(query)
    except Exception as e:
        print(e)

## Conclusion

Congratulations! You have succesfully connected an agent to LangSmith for debugging, graded it using multiple evaluators across a dataset, and created a new project for monitoring your agent in a simulated "production" application!

This was a quick guide to get started, but there are many more ways to use LangSmith to speed up your developer flow and produce better products.

For more information on how you can get the most out of LangSmith, check out [LangSmith documentation](https://docs.smith.langchain.com/docs/),

and please reach out with questions, feature requests, or feedback at [support@langchain.dev](mailto:support@langchain.dev).